In [ ]:
import pandas as pd

In [ ]:
# make sure all save have a centroid
# [x.save() for x in Site.objects.all()]
sites = Site.objects.all()

In [ ]:
# store cadastral_comunitey-lau2name for sites without polys
cc = []
for x in sites:
    hansi = {}
    hansi['site_id'] = x.id
    hansi['ccs'] = []
    for y in x.cadastral_community.all():
        hansi['ccs'].append(y.lau2_id)
    hansi['ccs'] = list(set(hansi['ccs']))
    cc.append(hansi)
cc[0]

In [ ]:
df = pd.DataFrame(cc)

In [ ]:
df.to_csv('sites_cc.csv')

In [ ]:
# copy sau_nam to sm_adm if this field is empty
for x in sites:
    if x.sm_adm:
        smlst = x.sm_adm
    else:
        smlst = "; ".join([y.saunam for y in x.cadastral_community.all()])
        if smlst:
            pass
        else:
            smlst = "; ".join([y.lau2nam for y in x.cadastral_community.all()])
    x.sm_adm = smlst
    x.save()
    print("Site: {}; smlst: {}".format(x.id, smlst))

In [ ]:
# delete current municipalities
Municipality.objects.all().delete()

In [ ]:
# import new shapes
from shapes.load import import_shapes
import_shapes()

In [ ]:
# hook sites to municipalities by spatial lookup deprecated in favor of lau2_id match
# for x in sites:
#     if x.centroid:
#         admus = Municipality.objects.filter(geom__contains=x.centroid)
#         x.cadastral_community.set(admus)

In [ ]:
lau_df = pd.read_csv('sites_cc.csv', converters={"ccs": lambda x: x.strip("[]").replace('\'', '')})
lau_df

In [ ]:
for i, row in lau_df.iterrows():
    site = Site.objects.get(id=row['site_id'])
    ccs = []
    for x in row['ccs'].split(','):
        lau_id = x.strip()
        if lau_id:
            try:
#                 print(Municipality.objects.get(lau2_id=x))
                ccs.append(Municipality.objects.get(lau2_id=lau_id))
            except Exception as e:
                print(e)
                print("FEHLER at Site: {};  LAU2_ID: {}".format(site.id, x.strip()))
    site.cadastral_community.set(ccs)